# I2C Communication TCA

In [8]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
from pynq import Overlay

base = BaseOverlay("base.bit")
# lib = MicroblazeLibrary(base.RPI, ['i2c'])
lib = MicroblazeLibrary(base.PMODB, ['i2c'])

In [9]:
from time import sleep

# TCA9548A
TCA_DEVICE_ADDRESS = 0x70

#  !!!! MPU MUST be supplied with 5V because of the onboard 3.3v regulator !!!!
MPU_DEVICE_ADDRESS = 0x68
MPU_PWR_MGMT_1   = 0x6B
MPU_SMPLRT_DIV   = 0x19
MPU_CONFIG       = 0x1A
MPU_GYRO_CONFIG  = 0x1B
MPU_INT_ENABLE   = 0x38
MPU_ACCEL_XOUT_H = 0x3B
MPU_ACCEL_YOUT_H = 0x3D
MPU_ACCEL_ZOUT_H = 0x3F
MPU_GYRO_XOUT_H  = 0x43
MPU_GYRO_YOUT_H  = 0x45
MPU_GYRO_ZOUT_H  = 0x47
MPU_SIGNAL_PATH_RESET = 0x68


## Pmod I2C Pin Definitions
PMOD_SDA_PIN = 6  ## or 2
PMOD_SCL_PIN = 7  ## or 3
# RPI_SDA_PIN = 3
# RPI_SCL_PIN = 5

# i2c_device = lib.i2c_open(RPI_SDA_PIN, RPI_SCL_PIN)
i2c_device = lib.i2c_open(PMOD_SDA_PIN, PMOD_SCL_PIN)

def iic_writeByte(devAddr, regAddr, data):
    temp = bytearray(2)
    temp[0] = regAddr
    temp[1] = data
    i2c_device.write(devAddr, temp, 2)
    
    
def iic_readByte(devAddr, regAddr):
    temp = bytearray(1)
    temp[0] = regAddr
    i2c_device.write(devAddr, temp, 1)
    i2c_device.read(devAddr, temp, 1)    
    return temp[0]


def TCA_channel_select(channel):
    """Select an individual channel."""
    if channel > 7:
        return
    iic_writeByte(TCA_DEVICE_ADDRESS, 0x00, 1 << channel) # TODO: what is the regAddr ??

    
    
# https://www.electronicwings.com/raspberry-pi/mpu6050-accelerometergyroscope-interfacing-with-raspberry-pi
def MPU_Init():    
    iic_writeByte(MPU_DEVICE_ADDRESS, MPU_SMPLRT_DIV, 7)
    sleep(100e-3)
    iic_writeByte(MPU_DEVICE_ADDRESS, MPU_PWR_MGMT_1, 1)
    sleep(100e-3)
    iic_writeByte(MPU_DEVICE_ADDRESS, MPU_CONFIG, 0)
    sleep(100e-3)
    iic_writeByte(MPU_DEVICE_ADDRESS, MPU_GYRO_CONFIG, 24)
    sleep(100e-3)
    iic_writeByte(MPU_DEVICE_ADDRESS, MPU_INT_ENABLE, 1)
    sleep(100e-3)
    


def MPU_read(addr):
    #Accelero and Gyro value are 16-bit
    high = iic_readByte(MPU_DEVICE_ADDRESS, addr)
    low = iic_readByte(MPU_DEVICE_ADDRESS, addr+1)

    #concatenate higher and lower value
    value = ((high << 8) | low)

    #to get signed value from mpu6050
    if(value > 32768):
            value = value - 65536
    return value

    
TCA_channel_select(1)
MPU_Init()
while True:
    #Read Accelerometer raw value
    acc_x = MPU_read(MPU_ACCEL_XOUT_H)
    acc_y = MPU_read(MPU_ACCEL_YOUT_H)
    acc_z = MPU_read(MPU_ACCEL_ZOUT_H)

    #Read Gyroscope raw value
    gyro_x = MPU_read(MPU_GYRO_XOUT_H)
    gyro_y = MPU_read(MPU_GYRO_YOUT_H)
    gyro_z = MPU_read(MPU_GYRO_ZOUT_H)

    #Full scale range +/- 250 degree/C as per sensitivity scale factor
    Ax = acc_x/16384.0
    Ay = acc_y/16384.0
    Az = acc_z/16384.0

    Gx = gyro_x/131.0
    Gy = gyro_y/131.0
    Gz = gyro_z/131.0

    print ("Gx=%.2f" %Gx, u'\u00b0'+ "/s", "\tGy=%.2f" %Gy, u'\u00b0'+ "/s", "\tGz=%.2f" %Gz, u'\u00b0'+ "/s", "\tAx=%.2f g" %Ax, "\tAy=%.2f g" %Ay, "\tAz=%.2f g" %Az) 	
    sleep(1)

i2c_device.close()  


Gx=-0.19 °/s 	Gy=-0.40 °/s 	Gz=0.50 °/s 	Ax=-0.02 g 	Ay=0.00 g 	Az=-1.03 g
Gx=-0.18 °/s 	Gy=-0.41 °/s 	Gz=0.47 °/s 	Ax=-0.02 g 	Ay=0.00 g 	Az=-1.01 g
Gx=-0.21 °/s 	Gy=-0.41 °/s 	Gz=0.49 °/s 	Ax=-0.02 g 	Ay=0.02 g 	Az=-1.01 g
Gx=-0.19 °/s 	Gy=-0.41 °/s 	Gz=0.49 °/s 	Ax=-0.02 g 	Ay=0.01 g 	Az=-1.01 g
Gx=-0.21 °/s 	Gy=-0.44 °/s 	Gz=0.51 °/s 	Ax=-0.03 g 	Ay=0.01 g 	Az=-1.01 g
Gx=-0.20 °/s 	Gy=-0.44 °/s 	Gz=0.49 °/s 	Ax=-0.02 g 	Ay=0.01 g 	Az=-1.01 g
Gx=-0.20 °/s 	Gy=-0.42 °/s 	Gz=0.51 °/s 	Ax=-0.02 g 	Ay=0.01 g 	Az=-1.01 g
Gx=-0.17 °/s 	Gy=-0.40 °/s 	Gz=0.49 °/s 	Ax=-0.03 g 	Ay=0.02 g 	Az=-1.01 g
Gx=-1.95 °/s 	Gy=-0.60 °/s 	Gz=0.43 °/s 	Ax=-0.00 g 	Ay=0.01 g 	Az=-1.03 g
Gx=0.94 °/s 	Gy=-0.73 °/s 	Gz=0.45 °/s 	Ax=-0.02 g 	Ay=-0.10 g 	Az=-0.97 g
Gx=0.42 °/s 	Gy=-0.23 °/s 	Gz=0.39 °/s 	Ax=-0.06 g 	Ay=-0.24 g 	Az=-1.05 g
Gx=0.20 °/s 	Gy=-0.39 °/s 	Gz=0.53 °/s 	Ax=-0.02 g 	Ay=-0.32 g 	Az=-0.94 g
Gx=0.21 °/s 	Gy=-0.37 °/s 	Gz=0.50 °/s 	Ax=-0.03 g 	Ay=-0.35 g 	Az=-0.95 g
Gx=-0.11 °/s 	Gy=-0.44 °/

KeyboardInterrupt: 

# MOTOR COMMUNICATION

In [10]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
from pynq import Overlay

base = BaseOverlay("base.bit")
# lib = MicroblazeLibrary(base.RPI, ['i2c'])
lib = MicroblazeLibrary(base.PMODB, ['i2c'])

In [32]:
from time import sleep
import math

# TCA9548A
TCA_DEVICE_ADDRESS = 0x70


## Pmod I2C Pin Definitions
PMOD_SDA_PIN = 6  ## or 2
PMOD_SCL_PIN = 7  ## or 3
# RPI_SDA_PIN = 3
# RPI_SCL_PIN = 5


# PCA9685
PCA9685_ADDRESS = 0x40
PCA9685_MODE1 = 0x00
PCA9685_MODE2 = 0x01
SUBADR1 = 0x02
SUBADR2 = 0x03
SUBADR3 = 0x04
PCA9685_PRESCALE = 0xFE
LED0_ON_L = 0x06
LED0_ON_H = 0x07
LED0_OFF_L = 0x08
LED0_OFF_H = 0x09

# Bits:
PCA9685_RESTART = 0x80
PCA9685_SLEEP = 0x10
PCA9685_EXTCLK = 0x40
PCA9685_ALLCALL = 0x01
PCA9685_INVRT = 0x10
PCA9685_OUTDRV = 0x04


# i2c_device = lib.i2c_open(RPI_SDA_PIN, RPI_SCL_PIN)
i2c_device = lib.i2c_open(PMOD_SDA_PIN, PMOD_SCL_PIN)

def iic_writeByte(devAddr, regAddr, data):
    temp = bytearray(2)
    temp[0] = regAddr
    temp[1] = data
    i2c_device.write(devAddr, temp, 2)
    
    
def iic_readByte(devAddr, regAddr):
    temp = bytearray(1)
    temp[0] = regAddr
    i2c_device.write(devAddr, temp, 1)
    i2c_device.read(devAddr, temp, 1)    
    return temp[0]


def TCA_channel_select(channel):
    """Select an individual channel."""
    if channel > 7:
        return
    iic_writeByte(TCA_DEVICE_ADDRESS, 0x00, 1 << channel)
    sleep(1)

# https://github.com/adafruit/Adafruit_Python_PCA9685/blob/master/Adafruit_PCA9685/PCA9685.py
def PCA_Init():    
    iic_writeByte(PCA9685_ADDRESS, PCA9685_MODE2, PCA9685_OUTDRV)
    iic_writeByte(PCA9685_ADDRESS, PCA9685_MODE1, PCA9685_ALLCALL)
    sleep(0.005)
    mode1 = iic_readByte(PCA9685_ADDRESS, PCA9685_MODE1)
    mode1 = mode1 & ~PCA9685_SLEEP 
    iic_writeByte(PCA9685_ADDRESS, PCA9685_MODE1, mode1)
    sleep(0.005)
    
    set_pwm_freq(1000)


def set_pwm_freq(freq_hz):
    prescaleval = 25000000.0    # 25MHz
    prescaleval /= 4096.0       # 12-bit
    prescaleval /= float(freq_hz)
    prescaleval -= 1.0
    prescale = int(math.floor(prescaleval + 0.5))
    oldmode = iic_readByte(PCA9685_ADDRESS, PCA9685_MODE1)
    newmode = (oldmode & 0x7F) | 0x10    # sleep
    iic_writeByte(PCA9685_ADDRESS, PCA9685_MODE1, newmode)  # go to sleep
    iic_writeByte(PCA9685_ADDRESS, PCA9685_PRESCALE, prescale)
    iic_writeByte(PCA9685_ADDRESS, PCA9685_MODE1, oldmode)
    sleep(0.005)
    iic_writeByte(PCA9685_ADDRESS, PCA9685_MODE1, oldmode | 0x80)
    
    

def set_pwm(channel, on, off):
    """Sets a single PWM channel."""
    iic_writeByte(PCA9685_ADDRESS, LED0_ON_L+4*channel, on & 0xFF)
    iic_writeByte(PCA9685_ADDRESS, LED0_ON_H+4*channel, on >> 8)
    iic_writeByte(PCA9685_ADDRESS, LED0_OFF_L+4*channel, off & 0xFF)
    iic_writeByte(PCA9685_ADDRESS, LED0_OFF_H+4*channel, off >> 8)

    
def arm_motors():
    channels = [3, 4, 7, 8]
    
     # Convert microseconds to PWM range used by PCA9685
    def us_to_pwm(value_us):
        return int(value_us * 4096 / 20000)  # 20ms period
    
    min_pwm = us_to_pwm(1000)
    max_pwm = us_to_pwm(2000)
    
    for channel in channels:
        sleep(0.5)
        set_pwm(channel, 0, min_pwm)
        sleep(1)
        set_pwm(channel, 0, max_pwm)
        sleep(1)
        set_pwm(channel, 0, min_pwm)
        sleep(0.5)
        
    print("ARMING SEQUENCE DONE")

print("TCA SELECT CHANNEL 0")
TCA_channel_select(0)

print("INITIALIZE PWM DRIVER")
PCA_Init()

# PCA9685 12 bit PWM resolution ==> max value = 4096
# ESC: Available throttle calibration range is from 1000us to 2000us

# i2c_device.close()  


TCA SELECT CHANNEL 0
INITIALIZE PWM DRIVER


In [36]:
print("STARTING ARMING SEQUENCE")
arm_motors()

STARTING ARMING SEQUENCE
ARMING SEQUENCE DONE


In [37]:
def start_motor(speed_us=1000):
    """
    Start the motors with the specified speed.
    
    Parameters:
    speed_us (int): Desired pulse width in microseconds for motor speed (typically between 1100µs and 1900µs).
    """
    channels = [3, 4, 7, 8]
    
    # Convert microseconds to PWM range used by PCA9685
    def us_to_pwm(value_us):
        return int(value_us * 4096 / 20000)  # 20ms period
    
    speed_pwm = us_to_pwm(speed_us)
    
    for channel in channels:
        set_pwm(channel, 0, speed_pwm)
    
    print("MOTORS STARTED WITH SPEED PWM VALUE: ", speed_pwm)

    
start_motor(1500)

MOTORS STARTED WITH SPEED PWM VALUE:  307


In [38]:
start_motor(0)

MOTORS STARTED WITH SPEED PWM VALUE:  0
